In [6]:
import requests
import json
import pandas as pd
from bls_definitions import bls_ppi_codes

In [7]:
key = "9648e84bbf0f4c38b01689279269a05e"
url = "https://api.bls.gov/publicAPI/v2/timeseries/data/"
startyear = "2004"
endyear = "2023"

In [8]:
selection_dict = {"seriesid": ['WPUFD4'], "startyear":startyear, "endyear":endyear, "registrationkey" : key}
selection = json.dumps(selection_dict)

In [9]:
headers = {'Content-type' : 'application/json'}

In [10]:
p = requests.post('https://api.bls.gov/publicAPI/v2/timeseries/data/', data=selection, headers=headers)

In [11]:
p = p.json()['Results']['series']

In [12]:
date_list = [f"{i['year']}-{i['period'][1:]}-01" for i in p[0]['data']]

In [13]:
df = pd.DataFrame(index = pd.to_datetime(date_list[::-1]))

In [14]:
for i in p:
    df[i['seriesID']] = pd.Series([j['value'] for j in i['data']]).astype(float).iloc[::-1].values


Attempt with first 50 codes from bls list

In [16]:
codes = list(bls_ppi_codes.keys())[:50]

In [17]:
selection_dict = {"seriesid": codes, "startyear":startyear, "endyear":endyear, "registrationkey" : key}
selection = json.dumps(selection_dict)

In [18]:
p = requests.post('https://api.bls.gov/publicAPI/v2/timeseries/data/', data=selection, headers=headers)

In [19]:
p

<Response [200]>

In [20]:
p = p.json()['Results']['series']

In [21]:
date_list = [f"{i['year']}-{i['period'][1:]}-01" for i in p[0]['data']]
# index = pd.to_datetime(date_list[::-1])

In [22]:
# df = pd.DataFrame()
data = {}
index = {}

In [23]:
for i in p:
    data[i['seriesID']] = pd.Series([j['value'] for j in i['data']]).astype(float).iloc[::-1].values
    index[i['seriesID']] = [f"{j['year']}-{j['period'][1:]}-01" for j in i['data']][::-1]


C:\Users\PETERC~1\AppData\Local\Temp/ipykernel_14524/2367738971.py:2: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  data[i['seriesID']] = pd.Series([j['value'] for j in i['data']]).astype(float).iloc[::-1].values


In [24]:
missing_codes = []
for i in p:
    if len(i['data']) == 0:
        missing_codes.append(i['seriesID'])


In [25]:
dfs = {}
for i in data.keys():
    dfs[i] = pd.DataFrame(data = data[i], index = pd.to_datetime(index[i]))

In [27]:
final = pd.concat(dfs,axis=1)

In [28]:
final.columns = final.columns.get_level_values(0)

In [29]:
final.to_csv('test.csv')